In [1]:
from pathlib import Path

In [5]:
DATA = Path('../tests/data').resolve()

# Sanitize

This is the process of curating a morphological file. It currently:

- removes zero-length segments
- raises if the morphology has no soma
- raises if the morphology has negative diameters

Note: more functionality may be added in the future

## Sanitize a single file

In [3]:
from neuror.sanitize import sanitize

sanitize(Path(DATA, 'simple-with-duplicates.asc'), 'data/output/sanitized.swc')

## Sanitize all files in a folder

In [4]:
from neuror.sanitize import sanitize_all

sanitize_all(DATA, 'data/output/sanitized')

100%|██████████| 27/27 [00:00<00:00, 149.48it/s]


# Cut plane repair

The cut plane repair aims at regrowing part of a morphologies that have been cut out
when the cell has been experimentally sliced.

## Repair a single file

In [5]:
from neuror.main import repair

repair(DATA / 'Neuron_slice.h5', 'data/output/repaired.h5')


Could not find apical point


## Repair all files in a folder

In [6]:
from neuror.full import repair_all

output = Path('data/output/repaired-folder')
output.mkdir(exist_ok=True)

repair_all(DATA / '../../examples/input-repair', output)

# Unravelling

Unravelling is the action of "stretching" the cell that has been shrunk because of the dehydratation caused by the slicing.

## Unravel a single file

In [7]:
from neuror.unravel import unravel

unravelled_neuron, mapping = unravel(DATA / 'Neuron_slice.h5')
unravelled_neuron.write('data/output/unravelled.h5')

# mapping contains the coordinates of each points of the morphology before/after unravelling
mapping

,x0,y0,z0,x1,y1,z1
0,-3.684218,10.765646,-13.829796,-3.684218,10.765646,-13.829796
1,-3.914217,11.355646,-14.289796,-3.786056,11.433075,-14.225735
2,-4.084218,11.915647,-14.529796,-3.869699,11.960962,-14.564023
3,-4.104218,12.465646,-14.839796,-3.959118,12.492048,-14.894101
4,-4.164217,12.855646,-15.079796,-4.017840,12.879383,-15.138690
...,...,...,...,...,...,...
17121,43.885784,143.625656,13.680204,49.387363,158.269501,16.787622
17122,44.525784,144.155655,13.750204,49.871132,158.948059,16.817884
17123,44.755783,144.475647,13.910204,50.117596,159.294205,16.836184
17124,45.095783,144.705643,13.910204,50.360828,159.624130,16.858242


## Unravel all morphologies in a folder

The following `unravel_all` function not only perform the unravelling of all the morphologies in the folder but also update their cut planes.

Because the unravelling distords morphologies, cut planes must be computed before correcting it.
`unravel_all` takes care of updating the position of the 'cut points' (points close to the cut plane) after they have been unravelled.
Note: the planes can be computed with the CLI: neuror cut-plane compute folder

The argument 'raw_planes' is a folder that must contain JSON files with cut planes for all morphology in the input folder. The name of the files should be the same as the morphology they refer to, but with the JSON extension.


In [8]:
from neuror.unravel import unravel_all

output = Path('data/output/unravelled-folder')
output.mkdir(exist_ok=True)

input_path = DATA / 'input-unravel-all'
raw_planes = input_path / 'raw-planes/'

unravel_planes = output / 'unravelled-planes'
unravel_planes.mkdir(exist_ok=True)

unravel_all(input_path, output, 
            raw_planes, unravel_planes)